In [1]:
from jinja2 import Template
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,get_response_synthesizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings,PromptTemplate
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core.retrievers import VectorIndexRetriever
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI

2025-05-21 21:28:55.002821: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 21:28:55.049194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 21:28:55.902951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
client = OpenAI(api_key="0",base_url="http://0.0.0.0:8000/v1")

In [3]:
# 格式化输出

class choice(BaseModel):
    """正确的选项"""

    correct_chice: str

In [24]:
prompt_choice='''
{{question}}
{{choice}}
'''
choice_template = Template(prompt_choice)

prompt_jianda='''
{{question}}
'''
jianda_template = Template(prompt_jianda)


extract_answer = '''
你是一个答案提取助手，请直接从我给出的包含答案选项和解析的文本中提取出正确答案选项,只返回选项对应的字母，不返回任何其他内容。

文本陈述:
{{chenshu}}
'''

jian_template = Template(extract_answer)

In [5]:
Settings.embed_model = HuggingFaceEmbedding(model_name="./BAAI/bge-m3",device='cuda:0')

In [6]:
documents = SimpleDirectoryReader(
    "./赛题制度文档",
    recursive=True
).load_data(show_progress=True)

Loading files:   0%|          | 0/895 [00:00<?, ?it/s]

In [7]:
Settings.llm = OpenAILike(
    model="Fin-R1",
    api_base='http://0.0.0.0:8000/v1',
    api_key='0',
    is_chat_model=True,
    temperature=0.6,
    top_p = 0.95,
    top_k = 20
    # additional_kwargs={
    #     "extra_body": {"enable_thinking": False},
    # },
)

index = VectorStoreIndex.from_documents(documents)

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=3,
)

In [8]:
response_synthesizer = get_response_synthesizer(
    
        response_mode="tree_summarize"
        # output_cls=choice
)

In [25]:
# 修改text_qa_template
new_choice_tmpl_str = (    "你是一个兼容监管制度知识专家。下面是上下文信息。\n"    "{context_str}\n"    "你将会再查询接受到一个金融监管制度方面不定项选择题和多个选项。请根据上下文信息和非先验知识, 在给出的选项中选出正确的答案（可能是单个或多个选项)"    "查询：{query_str}\n"    "答案:")
new_choice_tmpl = PromptTemplate(new_choice_tmpl_str)

new_jianda_tmpl_str = (    "你是一个兼容监管制度知识专家。下面是上下文信息。\n"    "{context_str}\n"    "你将会再查询接受到一个金融监管制度方面简答题。请根据上下文信息和非先验知识, 对问题做出正确的解答"    "查询：{query_str}\n"    "答案:")
new_jianda_tmpl = PromptTemplate(new_jianda_tmpl_str)

In [26]:
# query_engine = index.as_query_engine(
#         response_mode="tree_summarize",
#         output_cls=choice
# )

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.6)],
)

In [ ]:
data = pd.read_json('./testA.json',lines=True)
results = {
    "id":[],
    "answer":[]
}
for idx,row in data.iterrows():
    print("=====================================")
    if row['category']=='选择题':
        query_engine.update_prompts(
            {"response_synthesizer:summary_template": new_choice_tmpl}
        )
        prompt = choice_template.render(question=row['question'], choice = row['content'])
        response = query_engine.query(prompt)
        print(response)
        chenshu = jian_template.render(chenshu=response)
        message = [
        # {'role':"system","content":sys},
        {"role": "user", "content": chenshu}]
        result = client.chat.completions.create(messages=message, model="Fin-R1",
                                                temperature=0.1,
                                                top_p=0.8,
                                                max_tokens=1000,
                                                extra_body={
                                                    "repetition_penalty": 1.05,
                                                    "enable_thinking": False
                                                },)
        print(result.choices[0].message.content)
        result = [result.choices[0].message.content.strip()]
        results["id"].append(row["id"])
        results["answer"].append(result)
    else:
        print("============================================")
        query_engine.update_prompts(
            {"response_synthesizer:summary_template":new_jianda_tmpl}
        )
        prompt = jianda_template.render(question=row['question'])
        response = query_engine.query(prompt)
        print(response)
        results["id"].append(row["id"])
        results["answer"].append(result)



根据提供的文档内容，消费金融公司需要在《金融机构大额交易和可疑交易报告管理办法（2016修订）》施行前完成大额交易报告主体资格申请。该办法自**2017年7月1日**起施行（见文件第三十条）。因此，消费金融公司需在**2017年6月30日**前完成相关申请，以确保符合新法规的要求。

**答案: C. 2017年6月30日**


C


根据中国银监会办公厅2010年发布的《关于进一步提高两岸银行业相互开放水平有关工作的通知》（银监办发〔2010〕399号）规定，台湾银行在大陆的营业性机构申请经营人民币业务需满足以下条件：  
- **提出申请前在大陆开业2年以上**；  
- **提出申请前1年盈利**。  

因此，正确答案是 **C. 开业2年且最近1年盈利**。  

**答案: C**


C


In [ ]:
# local usage
embedding = OpenAIEmbedding().get_text_embedding("hello world")
embeddings = OpenAIEmbedding().get_text_embeddings(
    ["hello world", "hello world"]
)

# per-index
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)